In [53]:
import numpy as np
import copy
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [54]:
# # create data set
# n = 400
# _mus = np.array([[0,4], [-2,0]])
# _sigmas = np.array([[[3, 0], [0, 0.5]], [[1,0],[0,2]]])
# _pis = np.array([0.6, 0.4])
# X = np.concatenate([np.random.multivariate_normal(mu, sigma, int(pi*n))
#                     for pi, mu, sigma in zip(_pis, _mus, _sigmas)])

In [63]:
n=200
#number of samples per class: n
k=2
#number of classes: k
d=20
#dimensionality of samples: d
mus=np.random.random((k,d))*20-10 
#mean of the multivariate normal distribution | mus[i,j] is the mean for the ith cluster at the jth dimension
sigma=[np.eye(d) for i in range(k)]
#standard deviations are all assumed to be one. As the covariance matrix is identity the dimensions can be sampled independently
Pc=np.ones(k)/k
#setting priors: Pc

#Data Generation
X=[]
for i in range(n):
    for j in range(k):
#for each class generate n tuples of size d, taken from the distribution N(mus[j],1)
        point=[mean+np.random.randn() for mean in mus[j]]
        X.append(point)
#we have our dataset, where each point is a d+1 dimensional tuple where the last position represents its class 

X = np.asarray(X)

In [64]:
def w_mat(i,j):
    a = multivariate_normal.pdf(\
            X[index_matrix[i][j][0]],\
            mean=new_mus[index_matrix[i][j][1]],\
            cov=new_cov[index_matrix[i][j][1]],\
            allow_singular=True\
        )
    b = new_priors[index_matrix[i][j][1]]
    return a*b

def tabulate(x, y, f):
   """Return a table of f(x, y)."""
   #* is to unpack the two arrays which results after meshing
   return np.vectorize(f)( * np.meshgrid(x, y))

index_matrix = np.empty((X.shape[0],k),dtype=(int,2))
for i in range(index_matrix.shape[0]):
    for j in range(index_matrix.shape[1]):
        index_matrix[i][j] = (i,j)

def covid(n,m):
    co_vid = np.zeros((X.shape[1],X.shape[1]),dtype=float)
    for j in range(n):
        co_vid += W[m][j] * (((X[j] - new_mus[m]).T) @ (X[j] - new_mus[m]))
    return co_vid

In [65]:
d = X.shape[1]
n = X.shape[0]
# old_mus = np.array([np.random.rand(d) for _ in range(k)])
# random_rows = np.random.choice(X.shape[0], size=k, replace=False)
# old_mus = copy.deepcopy(X[random_rows, :])
old_mus = np.random.random((k,d))*20-10 
# old_mus = np.random.random((k,d))*np.max(X)-np.min(X)
new_mus = copy.deepcopy(old_mus)
print('original',mus)
print("initial", new_mus)

old_cov = np.asarray([np.eye(d) for _ in range(k)])
new_cov = copy.deepcopy(old_cov)

old_priors = np.full((k),1/k)
new_priors = copy.deepcopy(old_priors)

eps = 1e-7
t = 0
obt_eps = 1
while (obt_eps > eps) or (t==0):
    t += 1

    #unnormalised
    W_temp = copy.deepcopy(tabulate(list(range(X.shape[0])),list(range(k)),w_mat))
    temp = copy.deepcopy(W_temp)
    #normalised
    W = copy.deepcopy(temp/temp.sum(axis=0))
    # W = copy.deepcopy(W_temp)

    #get sum of W for each cluster
    temp = copy.deepcopy(W)
    sum_w = copy.deepcopy(temp.sum(axis=1))


    old_mus = copy.deepcopy(new_mus)
    #unnormalised
    new_mus_temp = copy.deepcopy(W @ X)
    temp = copy.deepcopy(new_mus_temp)
    #normalised
    new_mus = copy.deepcopy(temp/sum_w[:,None])
    # print(t,new_mus)

    old_cov = copy.deepcopy(new_cov)
    new_cov_temp = copy.deepcopy([covid(n,a) for a in range(k)])
    temp = copy.deepcopy(new_cov_temp)
    new_cov = copy.deepcopy([temp[a]/sum_w[a] for a in range(k)])

    old_priors = copy.deepcopy(new_priors)
    #normalised
    new_priors = copy.deepcopy(sum_w/n)
    obt_eps = np.sum([np.linalg.norm(new_mus[a]-old_mus[a]) for a in range(k)])
    # print("eps2",meh)

print("final", t,mus)


original [[ 9.3245812   0.5296101  -9.59451463  2.90523913  7.72276314 -7.03474969
  -4.43897938 -9.23632433  1.91075747 -3.32392836  4.77153665 -9.04291385
  -7.28017799 -7.69842459 -5.97750666  1.12609287 -3.00322687 -4.50392705
  -3.535464    4.02610131]
 [ 1.93188837 -2.21976769 -0.17026397 -8.82440586  4.83892254 -7.46240162
  -2.80778848  4.43408338  0.33350465  7.47506202  3.63597541  0.64503608
  -5.06098391 -6.82372625 -6.186499    8.0411551   0.80941609  2.21286556
  -1.59051185  3.89947242]]
initial [[-9.97037695 -7.21373906 -3.3346758   2.79657627 -0.23382936  2.84576267
   3.53766019 -5.88931316  0.5341006   2.58665388  5.25635066 -2.55223427
   1.02799173 -0.32911589  3.38927196 -3.61101785  0.01985154 -1.66840223
  -1.75980248  7.82593534]
 [-1.6382292  -9.1761404  -3.23233198  1.41536052  4.28643041 -4.76309466
  -0.0330121   4.22555311  6.92729808  3.66943127 -8.33436951  4.5909301
   2.8130274  -5.27306773 -6.35157472  6.56069929 -9.85399015  3.79680416
   1.42822165 

In [66]:
# print('Original means of the data for class 1 are: {} and the predicted means are {}'.format(mus[0],means[0]))
# print('Original means of the data for class 2 are: {} and the predicted means are {}'.format(mus[1],means[1]))
# print('The Original covariance matrix is an Identity matrix for both the classes.')
# print('Predicted covariance matrices of the data for class 1 is: \n {} \n and for class 2 is \n {}'.format(sigmas[0],sigmas[1]))
